In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/srilanka-geo/dristrict_aditional_data.csv
/kaggle/input/srilanka-geo/Grama_geo.json
/kaggle/input/srilanka-geo/grama_niladari_aditional_data.csv
/kaggle/input/srilanka-geo/Divisional_geo.json
/kaggle/input/srilanka-geo/District_geo.json
/kaggle/input/srilanka-geo/ds_aditional_data.csv


# Interactive choropleth maps  with Python and Folium for Srilanka (Divisional secretariat level) 

In [2]:
#Let's import libraries we need
import pandas as pd
import geopandas as gpd
import folium

In [3]:
# importing geo json files 
dsg=gpd.read_file('/kaggle/input/srilanka-geo/Divisional_geo.json')

In [4]:
# loading first two rows of the data set
dsg.head(2)

,ADM3_EN,ADM3_SI,ADM3_TA,ADM3_PCODE,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,ADM3ALT1SI,ADM3ALT2SI,ADM3ALT1TA,...,ADM0_EN,ADM0_SI,ADM0_TA,ADM0_PCODE,Shape_Leng,Shape_Area,date,validOn,validTo,geometry
0,[unknown],[unknown],[unknown],LK4100,,,,,,,...,Sri Lanka,ශ්‍රී ලංකා,இலங்கை,LK,0.041858,0.000077,2019-03-04T00:00:00,2020-03-05T00:00:00,1899-11-30T00:00:00,"POLYGON ((80.26160 9.62322, 80.25787 9.62729, ..."
1,[unknown],[unknown],[unknown],LK5100,,,,,,,...,Sri Lanka,ශ්‍රී ලංකා,இலங்கை,LK,0.436271,0.000715,2019-03-04T00:00:00,2020-03-05T00:00:00,1899-11-30T00:00:00,"MULTIPOLYGON (((81.65337 7.48736, 81.65711 7.4..."


In [5]:
#removing other data except "ADM3_PCODE" and "geometry"
dsg=dsg[['ADM3_PCODE','geometry']]

In [6]:
# loading aditional data
ds_a=pd.read_csv('/kaggle/input/srilanka-geo/ds_aditional_data.csv')
ds_a.rename(columns={'admin3Pcode':'ADM3_PCODE'}, inplace=True)
ds_a.head()

,admin3Name_en,admin3Name_si,ADM3_PCODE,risk_level,patients,population
0,[unknown],[unknown],LK4100,1,5,5000
1,[unknown],[unknown],LK5100,2,10,10000
2,[unknown],[unknown],LK6000,3,15,15000
3,Addalaichenai,අඩ්ඩාලච්චේන,LK5233,4,20,20000
4,Agalawatta,අගලවත්ත,LK1333,5,25,25000


In [7]:
divisional=dsg.merge(ds_a,on="ADM3_PCODE")
divisional.head()

,ADM3_PCODE,geometry,admin3Name_en,admin3Name_si,risk_level,patients,population
0,LK4100,"POLYGON ((80.26160 9.62322, 80.25787 9.62729, ...",[unknown],[unknown],1,5,5000
1,LK5100,"MULTIPOLYGON (((81.65337 7.48736, 81.65711 7.4...",[unknown],[unknown],2,10,10000
2,LK6000,"POLYGON ((79.82493 7.78566, 79.82307 7.79201, ...",[unknown],[unknown],3,15,15000
3,LK5233,"POLYGON ((81.86513 7.30019, 81.86147 7.30050, ...",Addalaichenai,අඩ්ඩාලච්චේන,4,20,20000
4,LK1333,"POLYGON ((80.18598 6.58012, 80.18274 6.58095, ...",Agalawatta,අගලවත්ත,5,25,25000


In [10]:
# Defining the style function and Highlight function 

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



# declaring the map with starting point
m = folium.Map(location=[7.8731, 80.7718], zoom_start=8)

choropleth=folium.Choropleth(
    geo_data=divisional,
    name='choropleth',
    data= divisional,
    columns=['ADM3_PCODE', 'risk_level'],
    key_on='properties.ADM3_PCODE',
    fill_color='RdPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    
    legend_name='Covid-19 risk Level with dummy data',

).add_to(m)

# adding some geo loacation (location pins )

folium.Marker(
    location=[6.905655, 79.927357],
    popup='Point one',
    
    icon=folium.Icon(icon='cloud')
).add_to(m)

folium.Marker(
    location=[6.7106,79.9074],
    popup='Point two',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[7.9403,81.01886823],
    popup='Point three',
    icon=folium.Icon(color='blue', icon='bar-chart')
).add_to(m)

folium.Marker(
    location=[9.3803, 80.3770],
    popup='Point four',
    icon=folium.Icon(color='red', icon='university')
).add_to(m)

#adding the same geo layer again on top of previouse layer to make interactivity 

INT = folium.features.GeoJson(
    divisional,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['admin3Name_en','patients','population'],
        aliases=['Divisionl sec: ','Numer of Patients:','Population'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(INT)
m.keep_in_front(INT)
folium.LayerControl().add_to(m)


In [11]:
m
m.save('divisional_final.html')
